# Exercise 2

**Please Note**: We updated the requirements.txt

Please install the new requirements before editing this exercise.

## Import packages

In [106]:
import os

from vll.utils.download import download_mnist
import numpy as np
import matplotlib.pyplot as plt

import skimage
import skimage.io

import torch
import torch.nn.functional as F
from torchvision import transforms

from models.mnist.simple_cnn import Net

## Task 1
(2 points)

In this task, you will learn some basic tensor operations using the PyTorch library.

Reference for torch: https://pytorch.org/docs/stable/torch.html

In [107]:
# Create a numpy array that looks like this: [0, 1, 2, ..., 19]
arr = np.array(np.arange(20))

# Convert the numpy array to a torch tensor
tensor = torch.from_numpy(arr)
#tensor = torch.tensor(arr, dtype=torch.float32)
#tensor = torch.tensor(arr)
print(tensor)

# Create a tensor that contains random numbers.
# It should have the same size like the numpy array.
# Multiply it with the previous tensor.
rand_tensor = torch.rand(tensor.shape)
tensor = torch.mul(tensor, rand_tensor)
print(tensor)

# Create a tensor that contains only 1s.
# It should have the same size like the numpy array.
# Substract it from the previous tensor.
one_tensor = torch.ones(tensor.shape)
tensor = torch.sub(tensor, one_tensor)
print(tensor)

# Get the 5th element using a index.
element = tensor[4]
print(element)

# Create a tensor that contains only 0s.
# It should have the same size like the numpy array.
# Multiply it with the previous tensor without any assignment (in place).
zero_tensor = torch.zeros(tensor.shape)
tensor.mul_(zero_tensor)


tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19], dtype=torch.int32)
tensor([ 0.0000,  0.9161,  1.0114,  2.6853,  1.2015,  2.4793,  0.4979,  1.0351,
         5.1272,  6.2887,  8.6539,  7.7728,  6.3841, 10.1412,  4.4647,  4.6444,
        10.4527, 13.0081, 16.5515, 14.6548])
tensor([-1.0000e+00, -8.3930e-02,  1.1431e-02,  1.6853e+00,  2.0151e-01,
         1.4793e+00, -5.0206e-01,  3.5090e-02,  4.1272e+00,  5.2887e+00,
         7.6539e+00,  6.7728e+00,  5.3841e+00,  9.1412e+00,  3.4647e+00,
         3.6444e+00,  9.4527e+00,  1.2008e+01,  1.5551e+01,  1.3655e+01])
tensor(0.2015)


tensor([-0., -0., 0., 0., 0., 0., -0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [108]:
# Load the image from the last exercise as RGB image.
origin = skimage.io.imread(r"data/pepo.jpg")
image = origin.copy()
print(f"Original shape: {image.shape}")

# Convert the image to a tensor
transform = transforms.ToTensor()     #Encodes the pixel values to a range of [0, 1] 
image = transform(image)

# Print its shape
print(f"Convert the image to a tensor: {image.shape}")

# Flatten the image
image = torch.flatten(image)
print(f"Flatten the image: {len(image)}")

# Add another dimension resulting in a 1x78642 tensor
image = torch.unsqueeze(image, dim=0)
print(f"Add another dimension resulting in a 1x78642 tensor: {image.shape}")

# Revert the last action
image = torch.squeeze(image, dim=0)
print(image.shape)

# Reshape the tensor, so that it has the original 2D dimensions
image = image.view(origin.shape)
print(f"Reshape the tensor to origin: {image.shape}")

# Calculate the sum, mean and max of the tensor
print(f"Sum: {torch.sum(image)}")
print(f"Mean: {torch.mean(image)}")
print(f"Max: {torch.max(image)}")

Original shape: (512, 512, 3)
Convert the image to a tensor: torch.Size([3, 512, 512])
Flatten the image: 786432
Add another dimension resulting in a 1x78642 tensor: torch.Size([1, 786432])
torch.Size([786432])
Reshape the tensor to origin: torch.Size([512, 512, 3])
Sum: 358450.375
Mean: 0.45579323172569275
Max: 1.0


## Task 2
(2 points)

Use Autograd to perform operations on a tensor and output then gradients.

In [109]:
# Create a random 2x2 tensor which requires gradients
x = torch.randn(2, 2, requires_grad=True)
print(x)

# Create another tensor by adding 2.0
y = torch.add(x, 2.0)
print(y)

# Create a third tensor z = y^2
z = torch.pow(y, 2)
print(z)

# Compute out as the mean of values in z
out = torch.mean(z)
print(out)

# Perform back propagation on out
out.backward()

# Print the gradients dout/dx
print(x.grad)

# Create a copy of y whithout gradients
y2 = y.detach()
print(y2.requires_grad)

# Perform the mean operation on z
# with gradients globally disabled
with torch.no_grad():
    print(torch.mean(z))

tensor([[0.5407, 0.9547],
        [0.3617, 0.1021]], requires_grad=True)
tensor([[2.5407, 2.9547],
        [2.3617, 2.1021]], grad_fn=<AddBackward0>)
tensor([[6.4554, 8.7300],
        [5.5775, 4.4187]], grad_fn=<PowBackward0>)
tensor(6.2954, grad_fn=<MeanBackward0>)
tensor([[1.2704, 1.4773],
        [1.1808, 1.0510]])
False
tensor(6.2954)


## Task 3
(3 points)

Implement a Dataset class for MNIST.

In [25]:
# We first download the MNIST dataset
download_mnist()

Extract mnist.tar.gz
Done!


In [110]:
from skimage import img_as_float32

class MNIST:
    """
    Dataset class for MNIST
    """

    def __init__(self, root=r"data/mnist/testing", transform=None):
        """
        root -- path to either "training" or "testing"
        
        transform -- transform (from torchvision.transforms)
                     to be applied to the data
        """
        # save transforms
        self.root_dir = root
        self.transform = transform
        self.samples=[]
        
        # TODO: create a list of all subdirectories (named like the classes) 
        #       within the dataset root
        class_dirs = [entry.name for entry in os.scandir(root) if entry.is_dir()]
        # TODO: create a list of paths to all images
        #       with the ground truth label
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(sorted(class_dirs))}
        for class_name in class_dirs:
            class_path = os.path.join(root, class_name)
            for entry in os.scandir(class_path):
                if entry.is_file() and entry.name.endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append((entry.path, self.class_to_idx[class_name]))        
    
    
    def __len__(self):
        """
        Returns the lenght of the dataset (number of images)
        """
        # TODO: return the length (number of images) of the dataset
        return len(self.samples)

    def __getitem__(self, index):
        """
        Loads and returns one image as floating point numpy array
        
        index -- image index in [0, self.__len__() - 1]
        """
        # TODO: load the ith image as an numpy array (dtype=float32)
        path, label = self.samples[index]
        img = img_as_float32(skimage.io.imread(path))  # (512, 512, 3)
        
        # TODO: apply transforms to the image (if there are any)
        if self.transform:
            img =  self.transform(img)

        # TODO: return a tuple (transformed image, ground truth)
        return img, label

## Task 4
(3 points)

You can now load a pretrained neural network model we provide.
Your last task is to run the model on the MNIST test dataset, plot some example images with the predicted labels and compute the prediction accuracy.

d:\conda\anaconda3\envs\3dcv\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
d:\conda\anaconda3\envs\3dcv\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
d:\conda\anaconda3\envs\3dcv\lib\site-pa

Accuracy: 98.00%


In [ ]:
def validate(model, data_loader):
    # TODO: Create a 10x10 grid of subplots
    fig, axes = plt.subplots(10, 10, figsize=(12, 12))
    
    model.eval()
    correct = 0 # count for correct predictions
    
    with torch.no_grad():
        for i, item in enumerate(data_loader):
            # TODO: unpack item into image and ground truth
            #       and run network on them
            image, label = item
            output = model(image)

            # TODO: get class with highest probability
            _, prediction = torch.max(output, 1)
            
            # TODO: check if prediction is correct
            #       and add it to correct count
            correct += prediction.eq(label).sum().item()
            
            # plot the first 100 images
            if i < 100:
                # TODO: compute position of ith image in the grid
                row, col = divmod(i,10)
                
                # TODO: convert image tensor to numpy array
                #       and normalize to [0, 1]
                image = image.squeeze().numpy()
                image_norm = (image - image.min()) / (image.max() - image.min())  # (28, 28)
                rgb_image = np.stack([image_norm]*3, axis=-1)
                
                # TODO: make wrongly predicted images red
                if prediction != label:
                    rgb_image[..., 0] = 1  # Setting the red channel full to make the image red
                # TODO: disable axis and show image
                axes[row, col].imshow(rgb_image)
                axes[row, col].axis('off')
                # TODO: disable axis and show image
                axes[row, col].imshow(rgb_image)
                axes[row, col].axis('off')
                
                # TODO: show the predicted class next to each image
                axes[row, col].set_title(f"Pred: {prediction.item()}", fontsize=8, color='red' if prediction != label else 'black')
                
            elif i == 100:
                plt.tight_layout()
                plt.show()
                break
                
    # TODO: compute and print the prediction accuracy in percent
    accuracy = (correct / i) * 100 
    print(f"Accuracy: {accuracy:.2f}%")

# create a DataLoader using the implemented MNIST dataset class
data_loader = torch.utils.data.DataLoader(
    MNIST('data/mnist/testing',
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=1, shuffle=True)

# create the neural network
model = Net()

# load the statedict from 'models/mnist/simple_cnn.pt'
model.load_state_dict(torch.load('models/mnist/simple_cnn.pt'))

# validate the model
validate(model, data_loader)